In [1]:
normal_set = set(["a", "b","c"]) 
  
print("Normal Set") 
print(normal_set) 


Normal Set
{'c', 'a', 'b'}


In [30]:
import numpy as np
print(np.ceil(np.logspace(0, 2.7, 30)).astype(int))

[  1   2   2   2   3   3   4   5   6   7   9  11  14  17  21  25  31  39
  48  59  73  91 112 139 172 213 264 327 405 502]


In [ ]:
    TRIALS = 5#DOING 5 TRIALS
    acc_scores = []#saving the accuracy for each trial on test set
    auc_scores = []#saving the roc auc for each trial on test set
    f1_scores = []#saving the f1 score for each trial on test set
    
    for i in range(TRIALS):
        #sampling 5000 training size for k-fold
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=5000)
        best_model = clf.fit(X_train, y_train)

        #top 3 best params model parameters
        model_acc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_accuracy'])]
        model_auc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_roc_auc_ovr'])]
        model_f1 = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

        model_params = [model_acc, model_auc, model_f1]

        #gets the scores for each model and append them to corresponding array
        for index, params in enumerate(model_params):
            k = params['n_neighbors']
            weight = params['weights']

            #training best models on validation set
            KNN_model = KNeighborsClassifier()
            KNN_model.set_params(n_neighbors=k, weights=weight)
            KNN_model.fit(X_train, y_train)#training on 5000 points

            #model == model_acc
            if index == 0:
                score = KNN_model.score(X_test, y_test)
                acc_scores.append(score)
            
            #model_auc
            elif index == 1:
                roc_score = roc_auc_score(y_test, KNN_model.predict_proba(X_test)[:, 1])
                auc_scores.append(roc_score)

            #appending to f1
            else:
                y_predict = KNN_model.predict(X_test)#predictions on test set
                f_score = f1_score(y_test, y_predict)
                f1_scores.append(f_score)
    
    return acc_scores, auc_scores, f1_scores